### 시계열 예측하기
각 타임 스텝 별 하나 이상의 값을 가진 시퀀스 데이터를 `시계열 데이터`라고 부르고, 이는 단변량 시계열과 다변량 시계열로 분류된다.
- 단변량 시계열 : 웹사이트의 시간 당 접속 사용자 수, 도시의 날짜별 온도
- 다변량 시계열 : 회사의 수입, 회사의 부채

두 자료형의 차이점은 타입 스텝 하나에 몇 개의 데이터들이 할당되냐의 차이가 있다. 단변량 시계열의 예시들은 한 스텝에 하나의 값을 가지는 반면, 다변량 시계열의 예시들은 한 스텝에 여러 가지의 변수들이 존재한다. 

In [1]:
import numpy as np

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1) # (4, bat_size, 1) 크기의 표준 정규 분포를 따르는 배열들 생성
    time = np.linspace(0, 1, n_steps)                                 # 0부터 1 사이를 n_steps의 크기로 분할
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))       # 사인함수 생성
    series += 0.2 * np.sin((time - offset2) * (freq2 * 20 + 20))       
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)       # 노이즈
    return series[..., np.newaxis].astype(np.float32)                 # 새로운 차원을 추가하고, float32형으로 형전환 하여 반환

위 함수는 batch_size 매개변수로 요청한 만큼 n_steps 길이의 여러 시계열을 만듭니다. 각 시계열에는 타임 스텝마다 하나의 값만 있습니다. 즉, 모든 시계열 데이터는 단변량 시계열 데이터입니다. 이 함수는 [배치 크기, 타임 스텝 수, 1] 크기의 넘파이 배열을 반환합니다. 각 시계열은 진폭이 같고 진동 수와 위상이 랜덤한 두 개의 사인 곡선을 더하고 약간의 노이즈를 추가합니다.

In [2]:
# 위에서 정의한 함수를 활용해서 훈련, 검증세트를 생성합니다.
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000: , -1]

위에서 생성된 데이터들의 형상을 생각해보면, X_train : [7000, 50, 1]크기 일 것이고, X_valid : [2000, 50, 1]일 것이며, X_test : [1000, 50, 1]일 것입니다. 각 시계열 데이터는 단변량 시계열 데이터로, 타겟은 열 벡터입니다. 즉, y_train의 형상은 [7000, 1]크기일 것입니다.

### 기준 성능
모델을 빌드하기 전에, 기준 성능을 몇 개 준비하는 것이 좋습니다. 그렇지 않으면 실제 기본 기능은 별로이지만, 예측이 잘 진행되었다고 생각할 수 있기 때문입니다, 이를 테면 각 시계열의 마지막 값을 그대로 예측하는 것입니다. 이를 `순진한 예측`이라고 하며, 가끔 이 성능을 뛰어넘는 것이 매우 어렵습니다. 이 경우 평균제곱오차 값은 0.020입니다.

In [3]:
from keras.losses import mean_squared_error

y_pred = X_valid[:, -1]
print(np.mean(mean_squared_error(y_valid, y_pred)))

0.020672668


또 다른 간단한 방법은 완전 연결 네트워크를 사용하는 것입니다. 이 네트워크는 입력마다 1차원 특성 배열을 기대하기 때문에 Flatten층을 추가해야 합니다. 시계열 값의 선형 조합으로 예측하기 위해 간단한 선형 회귀 모델을 빌드합니다.

In [4]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape = [50, 1]))
model.add(Dense(1))

model.compile(loss = "mean_squared_error", optimizer = "adam")
model.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 1s 2ms/step - loss: 0.1692 - val_loss: 0.0779
Epoch 2/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0469 - val_loss: 0.0321
Epoch 3/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0248 - val_loss: 0.0199
Epoch 4/20
219/219 [==============================] - 1s 2ms/step - loss: 0.0173 - val_loss: 0.0149
Epoch 5/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0138 - val_loss: 0.0123
Epoch 6/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0119 - val_loss: 0.0108
Epoch 7/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0104 - val_loss: 0.0097
Epoch 8/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0093 - val_loss: 0.0086
Epoch 9/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0083 - val_loss: 0.0077
Epoch 10/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0075 - val_loss: 0.0070

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [6]:
y_pred = model.predict(X_test)
np.mean(mean_squared_error(y_test, y_pred))

32/32 [==============================] - 0s 836us/step


0.0041871904

MSE지표와 Adam 옵티마이저로 모델을 컴파일하여 20 에포크동안 훈련시켜 평가한 결과는 약 0.004의 MSE값을 얻을 수 있었습니다. 순진한 예측보다 훨씬 나은 결과를 얻었습니다.

### SimpleRNN
keras에서 제공하는 SimpleRNN으로 이 성능을 앞지를 수 있는지 확인합니다.

In [7]:
from keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(1, input_shape = [None, 1]))

위 모델의 형태가 가장 간단하게 만들 수 있는 RNN입니다. 하나의 뉴런으로 이루어진 단 하나의 층을 가진 구조입니다. 순환 신경망은 어떤 길이의 타임 스텝도 처리할 수 있기 때문에 입력 시퀀스의 길이를 지정할 필요가 없어 첫 번째 입력 차원을 `None`으로 지정하였습니다. 기본적으로 SimpleRNN 층은 tanh 활성화 함수를 사용합니다. 초기 상태 h<sub>(init)</sub>를 0으로 설정하고 첫 번째 타임 스텝 x<sub>(0)</sub>와 함께 하나의 순환 뉴런으로 전달합니다. 뉴런은 이 값의 가중치 합을 계산하고 tanh 활성화 함수를 적용하여 결과를 만들어 첫 번째 y<sub>(0)</sub>를 출력합니다. 기본 RNN에서는 이 출력이 새로운 상태 h<sub>0</sub>가 됩니다. 이 새로운 상태는 다음 입력값 x<sub>(1)</sub>과 함께 동일한 순환 뉴런으로 전달됩니다. 이 과정이 마지막 타임 스텝까지 반복되면 마지막 y<sub>49</sub>를 출력하게 됩니다.

In [8]:
# 위에서 생성한 간단한 형태의 RNN모델을 동일한 방법으로 컴파일하고, 훈련합니다.
model.compile(optimizer = "adam", loss = "mean_squared_error")
model.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 6s 27ms/step - loss: 0.1741 - val_loss: 0.1226
Epoch 2/20
219/219 [==============================] - 6s 26ms/step - loss: 0.1059 - val_loss: 0.0926
Epoch 3/20
219/219 [==============================] - 5s 25ms/step - loss: 0.0811 - val_loss: 0.0717
Epoch 4/20
219/219 [==============================] - 5s 24ms/step - loss: 0.0640 - val_loss: 0.0579
Epoch 5/20
219/219 [==============================] - 5s 25ms/step - loss: 0.0526 - val_loss: 0.0485
Epoch 6/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0447 - val_loss: 0.0417
Epoch 7/20
219/219 [==============================] - 6s 26ms/step - loss: 0.0389 - val_loss: 0.0365
Epoch 8/20
219/219 [==============================] - 6s 25ms/step - loss: 0.0343 - val_loss: 0.0325
Epoch 9/20
219/219 [==============================] - 6s 26ms/step - loss: 0.0308 - val_loss: 0.0293
Epoch 10/20
219/219 [==============================] - 5s 24ms/step - loss: 0.0278 - val_lo

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [10]:
y_pred = model.predict(X_test)
np.mean(mean_squared_error(y_test, y_pred))

32/32 [==============================] - 0s 5ms/step


0.014616024

동일한 방법으로 컴파일하여 지표를 책정한 결과, 순진한 예측보다는 낫지만 간단한 선형 모델을 앞지르지는 못합니다. 이는 파라미터의 개수 때문인데, 선형 모델은 각 뉴런에 대해 입력(타임 스텝)마다 하나의 파라미터를 가지고 편향이 있어, 총 51개의 파라미터가 있습니다. 반면, 기본 RNN의 경우 순환 뉴런은 입력과 은닉 상태 차원(기본 RNN에서는 층의 순환 뉴런 개수와 같다.)마다 하나의 파라미터를 가지고 편향이 있어 총 3개의 파라미터를 가집니다.

### 심층 RNN
위처럼 단순히 빌드하는 방식과는 달리, RNN은 셀을 여러 층으로 쌓는 것이 일반적입니다. 이렇게 만든 것을 `심층 RNN`이라고 합니다. 위 방식처럼 단순히 순환 신경망 층을 쌓기만 하면 되는 것이 아니라, 맨 마지막 순환 신경망 층을 제외한 나머지 순환 신경망 층에서는 `return_sequences = True`로 설정해 주어야 합니다. 그렇지 않으면 모든 타임 스텝에 대한 출력을 담은 3D 배열이 아니라 마지막 타임 스텝의 출력만 담은 2D 배열이 출력되고, 다음 순환 층이 3D 형태로 시퀀스를 받지 못하기 때문에 작동하지 못합니다.

In [11]:
model = Sequential()
model.add(SimpleRNN(20, return_sequences = True, input_shape = [None, 1]))
model.add(SimpleRNN(20))
model.add(Dense(1))

In [12]:
model.compile(optimizer = "adam", loss = "mean_squared_error")
model.fit(X_train, y_train, validation_data = (X_valid, y_valid), epochs = 20)

Epoch 1/20
219/219 [==============================] - 15s 65ms/step - loss: 0.0338 - val_loss: 0.0066
Epoch 2/20
219/219 [==============================] - 13s 62ms/step - loss: 0.0049 - val_loss: 0.0042
Epoch 3/20
219/219 [==============================] - 13s 59ms/step - loss: 0.0040 - val_loss: 0.0038
Epoch 4/20
219/219 [==============================] - 13s 58ms/step - loss: 0.0037 - val_loss: 0.0036
Epoch 5/20
219/219 [==============================] - 12s 57ms/step - loss: 0.0035 - val_loss: 0.0036
Epoch 6/20
219/219 [==============================] - 13s 57ms/step - loss: 0.0034 - val_loss: 0.0035
Epoch 7/20
219/219 [==============================] - 13s 60ms/step - loss: 0.0034 - val_loss: 0.0035
Epoch 8/20
219/219 [==============================] - 13s 59ms/step - loss: 0.0033 - val_loss: 0.0035
Epoch 9/20
219/219 [==============================] - 12s 57ms/step - loss: 0.0033 - val_loss: 0.0033
Epoch 10/20
219/219 [==============================] - 13s 59ms/step - loss: 0.003

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, None, 20)          440       
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 20)                820       
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,281
Trainable params: 1,281
Non-trainable params: 0
_________________________________________________________________


In [14]:
y_pred = model.predict(X_test)
np.mean(mean_squared_error(y_test, y_pred))

32/32 [==============================] - 0s 10ms/step


0.0027874717

드디어 선형 모델을 앞질렀습니다.

### 여러 타임 스텝 앞을 예측하기
현재까지는 바로 다음 스텝의 값만 예측했지만 타깃을 적절히 바꾸어 여러 타임 스텝 앞의 값을 손쉽게 예측할 수 있습니다.

그 첫 번째 방식으로는 이미 훈련된 모델을 기반으로 예측을 진행하고, 그 예측을 입력으로 추가하는 방식입니다.

In [15]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new

for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis = 1)

Y_pred = X[:, n_steps:]

1/1 [==============================] - 0s 21ms/step


In [16]:
np.mean(mean_squared_error(Y_new, Y_pred))

0.034685835

두 번째 방법으로는 모델을 수정하는 방법입니다. RNN을 훈련하여 다음 값 10개를 한꺼번에 예측을 진행하는 것이죠. 값 1개를 출력하는 것이 아닌, 10개를 한꺼번에 출력합니다.

In [21]:
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid =series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test =series[9000:, :n_steps], series[9000:, -10:, 0]

In [22]:
model = Sequential()
model.add(SimpleRNN(20, return_sequences = True, input_shape = [None, 1]))
model.add(SimpleRNN(20))
model.add(Dense(10))

model.compile(optimizer = "adam", loss = "mean_squared_error")

In [23]:
model.fit(X_train, Y_train, epochs = 20, validation_data = (X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 14s 60ms/step - loss: 0.0622 - val_loss: 0.0277
Epoch 2/20
219/219 [==============================] - 13s 61ms/step - loss: 0.0221 - val_loss: 0.0159
Epoch 3/20
219/219 [==============================] - 13s 61ms/step - loss: 0.0161 - val_loss: 0.0132
Epoch 4/20
219/219 [==============================] - 13s 58ms/step - loss: 0.0139 - val_loss: 0.0124
Epoch 5/20
219/219 [==============================] - 13s 58ms/step - loss: 0.0132 - val_loss: 0.0129
Epoch 6/20
219/219 [==============================] - 12s 57ms/step - loss: 0.0121 - val_loss: 0.0120
Epoch 7/20
219/219 [==============================] - 13s 59ms/step - loss: 0.0115 - val_loss: 0.0108
Epoch 8/20
219/219 [==============================] - 15s 68ms/step - loss: 0.0109 - val_loss: 0.0103
Epoch 9/20
219/219 [==============================] - 14s 62ms/step - loss: 0.0108 - val_loss: 0.0101
Epoch 10/20
219/219 [==============================] - 14s 65ms/step - loss: 0.010

In [24]:
X_new, Y_new = series[:, :n_steps], series[:, -10:, 0] 
y_pred = model.predict(X_new)

np.mean(mean_squared_error(Y_new, y_pred))

313/313 [==============================] - 3s 9ms/step


0.008470253

잘 훈련이 된 모습을 확인할 수 있습니다. 하지만 이는 개선의 여지가 더 보입니다. 마지막 타임 스텝에서만 다음 값 10개를 예측하도록 모델을 훈련하는 대신 모든 타임 스텝에서 다음 값 10개를 예측하도록 모델을 훈련할 수 있습니다. 다르게 말하면 seq-to-vec RNN을 seq-to-seq RNN으로 바꿀 수 있습니다. 

이렇게 전환하면 생기는 장점으로는 마지막 타임 스텝에서의 출력 뿐만 아니라 모든 타임 스텝에서 RNN출력에 대한 항이 손실에 포함된다는 것입니다. 즉, 더 많은 오차 그래디언트가 흐르게 되고, 시간에 따라서만 흐를 필요가 없습니다. 각 타임 스텝에서의 출력에서 그래디언트가 흐를 수 있습니다. 이는 훈련을 안정적으로 만들고 훈련 속도를 높입니다.

In [29]:
Y = np.empty((10000, n_steps, 10)) # 각 타겟은 10D 벡터의 시퀀스이다.
for step_ahead in range(1, 10 + 1):
    Y[:, :, step_ahead - 1] = series[:, step_ahead : step_ahead + n_steps, 0]

Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

이 모델을 seq-to-seq 모델로 바꾸려면 모든 순환 층에 `return_sequences = True`로 지정해야 합니다. 그 다음 모든 타임 스텝에서 출력을 Dense층에 적용해야 합니다. 이러한 사태를 대비해 keras에서는 `TimeDistributed`층을 제공합니다. 이 층은 다른 층(ex. Dense layer)을 감싸서 입력 시퀀스의 모든 타임 스텝에 이를 적용합니다. 각 타임 스텝을 별개의 샘플처럼 다루도록 입력의 크기를 바꾸어 이를 효과적으로 수행합니다. 즉, 입력을 [batch_size, n_steps, input_dim]에서 [batch_size * n_steps, input_dim]의 크기로 바꿉니다.

그 다음 Dense 층에 적용합니다. 마지막으로 출력 크기를 시퀀스로 되돌립니다.

In [30]:
from keras.layers import TimeDistributed

model = Sequential()
model.add(SimpleRNN(20, return_sequences = True, input_shape = [None, 1]))
model.add(SimpleRNN(20, return_sequences = True))
model.add(TimeDistributed(Dense(10)))

훈련하는 동안 많은 출력들이 필요하지만, 예측과 평가에는 마지막 타임 스텝의 출력만 사용됩니다. 훈련을 위해 모든 출력에 걸쳐 MSE를 계산했었습니다만, 평가를 위해서는 마지막 타임 스텝의 출력에 대한 MSE만을 계산하는 사용자 정의 함수를 정의합니다.

In [33]:
from keras.metrics import mean_squared_error
from keras.optimizers import Adam

def last_time_step_mse(Y_true, Y_pred):
    return mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

optimizer = Adam(learning_rate = 0.01)
model.compile(optimizer = optimizer, loss = "mse", metrics = [last_time_step_mse])

In [34]:
model.fit(X_train, Y_train, validation_data = (X_valid, Y_valid), epochs = 20)

Epoch 1/20
219/219 [==============================] - 14s 60ms/step - loss: 0.0191 - last_time_step_mse: 0.0075 - val_loss: 0.0219 - val_last_time_step_mse: 0.0102
Epoch 2/20
219/219 [==============================] - 13s 58ms/step - loss: 0.0179 - last_time_step_mse: 0.0064 - val_loss: 0.0182 - val_last_time_step_mse: 0.0058
Epoch 3/20
219/219 [==============================] - 13s 59ms/step - loss: 0.0179 - last_time_step_mse: 0.0064 - val_loss: 0.0174 - val_last_time_step_mse: 0.0057
Epoch 4/20
219/219 [==============================] - 13s 58ms/step - loss: 0.0179 - last_time_step_mse: 0.0063 - val_loss: 0.0178 - val_last_time_step_mse: 0.0060
Epoch 5/20
219/219 [==============================] - 13s 58ms/step - loss: 0.0179 - last_time_step_mse: 0.0066 - val_loss: 0.0173 - val_last_time_step_mse: 0.0057
Epoch 6/20
219/219 [==============================] - 13s 58ms/step - loss: 0.0176 - last_time_step_mse: 0.0063 - val_loss: 0.0181 - val_last_time_step_mse: 0.0061
Epoch 7/20
219/2